In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read data

item_categories = pd.read_csv('Data/item_categories.csv')
items = pd.read_csv('Data/items.csv')
shops = pd.read_csv('Data/shops.csv')
sales = pd.read_csv('Data/sales_train_v2.csv')
submission = pd.read_csv('Data/sample_submission.csv')
test = pd.read_csv('Data/test.csv')

In [3]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [4]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [6]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [7]:
submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [8]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [9]:
# Monthly sales

pair_sales = sales.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum()
pair_sales.head()

shop_id  item_id  date_block_num
0        30       1                 31.0
         31       1                 11.0
         32       0                  6.0
                  1                 10.0
         33       0                  3.0
Name: item_cnt_day, dtype: float64

In [10]:
print(pair_sales.index[2])

(0, 32, 0)


In [11]:
print(len(pair_sales))

1609124


In [12]:
df = pd.DataFrame({'shop_id': [i[0] for i in pair_sales.index],'item_id': [i[1] for i in pair_sales.index],\
                   'date_block_num': [i[2] for i in pair_sales.index],'item_count': pair_sales})

In [13]:
print(df.head())

                                date_block_num  item_count  item_id  shop_id
shop_id item_id date_block_num                                              
0       30      1                            1        31.0       30        0
        31      1                            1        11.0       31        0
        32      0                            0         6.0       32        0
                1                            1        10.0       32        0
        33      0                            0         3.0       33        0


In [14]:
print(df.shape)

(1609124, 4)


In [15]:
df.to_csv('monthly_sales.csv', index=False)

In [34]:
data = {}
for i in df['shop_id'].unique()[:2]:
    data[i] = {}
    for j in df[df.shop_id == i]['item_id'].unique()[:2]:
        data[i][j] = []
        for k in range(33):
            val = df[(df.shop_id == i) & (df.item_id == j) & (df.date_block_num == k)].date_block_num.values
            if val.any():
                data[i][j].append(val[0])
            else:
                data[i][j].append(0)            

In [35]:
print(data)

{0: {30: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 31: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, 1: {27: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 29: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}}


In [38]:
pair = df.groupby(['shop_id', 'item_id'])['item_count'].apply(list)
pair.head()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: 'shop_id' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: 'item_id' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


shop_id  item_id
0        30              [31.0]
         31              [11.0]
         32         [6.0, 10.0]
         33          [3.0, 3.0]
         35         [1.0, 14.0]
Name: item_count, dtype: object

In [43]:
lst = []
for ind in test.index:
    each = test.iloc[ind]
    print(each)
    d = {'shop_id': each.shop_id, 'item_id': each.item_id, 'data': pair[each.shop_id][each.item_id]}
    lst.append(d)
    print(lst)

ID            0
shop_id       5
item_id    5037
Name: 0, dtype: int64
[{'item_id': 5037, 'shop_id': 5, 'data': [1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 3.0, 1.0]}]
ID            1
shop_id       5
item_id    5320
Name: 1, dtype: int64


KeyError: 5320